#Augmented Perspective

This iPython Notebook runs the the Augmented Perspective experiment flow

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where your augmented perspective repo is checked out
# e.g. 'projects/augmented-perspective':
# if FOLDERNAME, current directory of the iPython notebook will be used
FOLDERNAME = 'CS231A/augmented-perspective'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/.shortcut-targets-by-id/1g15eqMB8VyviBypnBlOVm8H-tu-OksvO/augmented-perspective


Then, you should be able to run the following with no problem.

In [2]:
%load_ext autoreload
%autoreload 2

import importlib
import sys

# pre-import everything to save time
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import torch

!nvidia-smi 
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.empty_cache() 
print("Using {} for computation".format(device))

Mon Mar 14 01:49:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# try import project libraries
import augmented_perspective
import calibration
import depth_model

Run the depth model with GPU againsts assets/kitti2.png

In [4]:
importlib.reload(depth_model)
argv = ["runner.py", "--image_path", "assets/kitti2.png", "--device", device]
depth_model.run_depth_model(argv)
original_image = io.imread("assets/kitti2.png")
depth_image_after_monodepth2 = io.imread("outputs/kitti2_monodepth2_depth.png")
depth_map_after_monodepth2 = np.load("outputs/kitti2_monodepth2_depth.npy")
print("Depth Map Size After Monodepth:{}".format(depth_map_after_monodepth2.shape))
depth_image_after_boosting = io.imread("outputs/kitti2_boosting_depth.png")
depth_map_after_boosting = np.load("outputs/kitti2_boosting_depth.npy")
print("Depth Map Size After Boosting:{}".format(depth_map_after_boosting.shape))


before ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-a70086d0-cb26-4da2-bf8c-8ea78e401960.json']
after ['runner.py', '--image_path', 'assets/kitti2.png', '--device', 'cuda']
Namespace(boosting=True, device='cuda', image_files=['kitti2.png'], image_path='../../assets', model_name='mono_1024x320', monodepth2=True, output_path='../../outputs/')
-> Loading model from  models/mono_1024x320
   Loading pretrained encoder
   Loading pretrained decoder
Reading images from ../../assets
-> Predicting on 1 test images
input_image.shape:  torch.Size([1, 3, 320, 1024])
disp.shape:  torch.Size([1, 1, 320, 1024])
disp_resized.shape:  torch.Size([1, 1, 370, 1224])
scaled_disp.shape:  torch.Size([1, 1, 320, 1024])
[[4.9915996 5.091959  5.1896105 ... 3.1745822 3.1612873 3.1302292]
 [5.0030622 5.1008735 5.1998634 ... 3.168944  3.1556315 3.1273923]
 [5.007049  5.097818  5.19861   ... 3.1896892 3.1729252 3.1429288]
 ...
 [1.812802  1.815973

Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_main


start processing
processing image 0 : kitti2
	 wholeImage being processed in : 1920
 	 	 DEBUG| GPU THRESHOLD REACHED 1920 ---> 1568
Adjust factor is: 1.3008887614678901
Selecting patchs ...
Target resolution:  (1510, 4995, 3)
Dynamicly change merged-in resolution; scale: 0.24503311258278146
	 Resulted depthmap res will be : (370, 1224)
patchs to process: 29
	 processing patch 0 | [  2   2 329 329]
	 processing patch 1 | [ 73   2 329 329]
	 processing patch 2 | [143   2 329 329]
	 processing patch 3 | [214   2 329 329]
	 processing patch 4 | [284   2 329 329]
	 processing patch 5 | [355   2 329 329]
	 processing patch 6 | [425   2 329 329]
	 processing patch 7 | [496   2 329 329]
	 processing patch 8 | [107 107 259 259]
	 processing patch 9 | [177 107 259 259]
	 processing patch 10 | [248 107 259 259]
	 processing patch 11 | [319 107 259 259]
	 processing patch 12 | [389 107 259 259]


KeyboardInterrupt: ignored

In [ ]:
plt.rcParams.update({'font.size': 6})
f, ax = plt.subplots(1, 3, dpi=500)
ax[0].imshow(original_image)
ax[0].title.set_text('RGB')
ax[1].imshow(depth_image_after_monodepth2)
ax[1].title.set_text('Monodepth2')
ax[2].imshow(depth_image_after_boosting)
ax[2].title.set_text('Boosting')

In [ ]:
plt.rcParams.update({'font.size': 6})
f, ax = plt.subplots(1, 3, dpi=500)
ax[0].imshow(original_image)
ax[0].title.set_text('RGB')
ax[1].imshow(depth_map_after_monodepth2, cmap='gray')
ax[1].title.set_text('Monodepth2')
ax[2].imshow(depth_map_after_boosting, cmap='gray')
ax[2].title.set_text('Boosting')

Run the augmented perspective code againsts assets/kitti2.png

In [ ]:
importlib.reload(augmented_perspective)
argv = ["runner.py", "--image_path", "assets/kitti2.png", "--depth_map_path", "outputs/kitti2_monodepth2_depth.npy"]
augmented_perspective.run_augmented_perspective(argv)
reprojected_image_after_monodepth2 = io.imread("output_images/kitti2_monodepth2_depth_reprojected.png")

argv = ["runner.py", "--image_path", "assets/kitti2.png", "--depth_map_path", "outputs/kitti2_boosting_depth.npy"]
augmented_perspective.run_augmented_perspective(argv)
reprojected_image_after_boosting = io.imread("output_images/kitti2_boosting_depth_reprojected.png")


In [ ]:
plt.rcParams.update({'font.size': 6})
f, ax = plt.subplots(1, 3, dpi=500)
ax[0].imshow(original_image)
ax[0].title.set_text('RGB')
ax[1].imshow(reprojected_image_after_monodepth2)
ax[1].title.set_text('Monodepth2')
ax[2].imshow(reprojected_image_after_boosting)
ax[2].title.set_text('Boosting')

In [ ]:
def draw_histogram(ax, data):
    counts, bins = np.histogram(data)
    ax.hist(bins[:-1], bins, weights=counts)

f, ax = plt.subplots(1, 2, dpi=500)
draw_histogram(ax[0], depth_map_after_monodepth2)
ax[0].title.set_text('Monodepth2')
draw_histogram(ax[1], depth_map_after_boosting)
ax[1].title.set_text('Boosting')

In [ ]:
raw_depth_map_after_monodepth2 = np.load("outputs/kitti2_monodepth2_resized_disp.npy")
raw_depth_map_after_boosting = np.load("outputs/kitti2_boosting_depth_raw.npy")
inverse_depth_map_after_boosting = np.load("outputs/kitti2_boosting_depth_inverse.npy")

f, ax = plt.subplots(1, 3, dpi=500)
draw_histogram(ax[0], raw_depth_map_after_monodepth2)
ax[0].title.set_text('Monodepth2 Raw')
draw_histogram(ax[1], raw_depth_map_after_boosting)
ax[1].title.set_text('Boosting Raw')
draw_histogram(ax[2], inverse_depth_map_after_boosting)
ax[2].title.set_text('Boosting Inverse')

Now run this against all images in asset/ folder

In [ ]:
import os
importlib.reload(depth_model)
argv = ["runner.py", "--image_path", "assets/", "--device", device]
depth_model.run_depth_model(argv)